In [15]:
import pandas as pd
import json

# Load the data
with open("playerWiseData.json", "r") as player_file:
    player_data = json.load(player_file)

with open("processedData.json", "r") as venue_file:
    venue_data = json.load(venue_file)

# Function to calculate top performers
def get_top_performers(player_data):
    top_performers = []

    for player_id, data in player_data.items():
        if 'batting' in data:
            recent_matches = data['batting'].get('matches', [])
            total_recent_runs = sum([match['runs'] for match in recent_matches if match['stats_type'] == 'recent'])
            total_recent_balls = sum([match['balls'] for match in recent_matches if match['stats_type'] == 'recent'])
            if total_recent_balls > 0:
                strike_rate = (total_recent_runs / total_recent_balls) * 100
                if total_recent_runs > 100:
                    top_performers.append(
                        f"{data['name']} scored {total_recent_runs} runs in the last {len(recent_matches)} matches with a strike rate of {strike_rate:.2f}."
                    )

        if 'bowling' in data:
            recent_matches = data['bowling'].get('matches', [])
            total_recent_wickets = sum([match['wickets'] for match in recent_matches if match['stats_type'] == 'recent'])
            total_recent_overs = sum([match['overs'] for match in recent_matches if match['stats_type'] == 'recent'])
            if total_recent_overs > 0:
                economy_rate = sum([match['runs'] for match in recent_matches if match['stats_type'] == 'recent']) / total_recent_overs
                if total_recent_wickets > 5:
                    top_performers.append(
                        f"{data['name']} took {total_recent_wickets} wickets in the last {len(recent_matches)} matches with an economy rate of {economy_rate:.2f}."
                    )

    return top_performers

# Function to analyze venue stats
def analyze_pitch_behavior(venue_data):
    venue_stats = venue_data['match_summary']['venue_stats']
    pitch_behavior = ""

    for stat in venue_stats:
        if stat['innings'] == 'Overall':
            if stat['bowler_type'] == 'pace' and stat['runs_per_wicket'] < 35:
                pitch_behavior = "The pitch favors pacers with lower runs per wicket."
            elif stat['bowler_type'] == 'spin' and stat['runs_per_wicket'] < 40:
                pitch_behavior = "The pitch favors spinners with lower runs per wicket."

    return pitch_behavior

# Function to provide player insights based on venue
def get_venue_favorites(player_data):
    venue_favorites = []

    for player_id, data in player_data.items():
        if 'batting' in data:
            venue_matches = data['batting'].get('matches', [])
            total_venue_runs = sum([match['runs'] for match in venue_matches if match['stats_type'] == 'venue'])
            if total_venue_runs > 50:
                venue_favorites.append(
                    f"{data['name']} has scored {total_venue_runs} runs at this venue."
                )

    return venue_favorites

# Function to fetch head-to-head statistics
def get_head_to_head(venue_data):
    return venue_data['match_summary'].get('head_to_head_analysis', [])

# Function to generate specialists dynamically
def get_phase_specialists(player_data, phase):
    specialists = []

    for player_id, data in player_data.items():
        if 'bowling' in data:
            phase_stats = data['bowling'].get('phase_stats', {}).get(phase, {})
            wickets = phase_stats.get('wickets', 0)
            if wickets > 3:
                specialists.append(
                    f"{data['name']} picked {wickets} wickets in the {phase.lower()} phase."
                )

    return specialists


# Function to generate differential picks dynamically
def get_differential_picks(player_data):
    differential_picks = []

    for player_id, data in player_data.items():
        if 'batting' in data:
            recent_matches = data['batting'].get('matches', [])
            if recent_matches and len(recent_matches) > 5:
                avg_runs = sum([match['runs'] for match in recent_matches]) / len(recent_matches)
                if avg_runs > 20:
                    differential_picks.append(
                        f"{data['name']} could be a differential pick."
                    )

    return differential_picks

# Generate insights
def generate_insights(player_data, venue_data):
    insights = {
        # "topPerformers": get_top_performers(player_data),
        "pitchBehavior": analyze_pitch_behavior(venue_data),
        "venueFavorites": get_venue_favorites(player_data),
        "headToHead": get_head_to_head(venue_data),
        "playerAvailability": "No injury updates available.",
        "powerplaySpecialists": get_phase_specialists(player_data, 'powerplay'),
        "deathOverSpecialists": get_phase_specialists(player_data, 'death'),
        "captainPick": "MS Dhoni",
        "viceCaptainPick": "Ravindra Jadeja",
        "differentialPicks": get_differential_picks(player_data),
        "weatherConditions": "No weather updates available.",
        "predictedPlayingXI": ["Team A: Player1, Player2, ...", "Team B: Player3, Player4, ..."],
        "fantasyTips": [
            "Pick at least one all-rounder to maximize points.",
            "Focus on bowlers who bowl in powerplay and death overs.",
            "Choose top-order batsmen for high-scoring venues."
        ]
    }

    return insights

# Generate and display insights
insights = generate_insights(player_data, venue_data)
print(json.dumps(insights, indent=2))


{
  "pitchBehavior": "The pitch favors pacers with lower runs per wicket.",
  "venueFavorites": [
    "Jos Buttler has scored 145 runs at this venue.",
    "Vijay Shankar has scored 115 runs at this venue.",
    "Shahrukh Khan has scored 58 runs at this venue.",
    "Shubman Gill has scored 286 runs at this venue.",
    "Ruturaj Gaikwad has scored 118 runs at this venue.",
    "Sai Sudharsan has scored 259 runs at this venue."
  ],
  "headToHead": [
    "Chennai Super Kings won by 63 runs",
    "Chennai Super Kings won by 5 wickets",
    "Chennai Super Kings won by 15 runs",
    "Gujarat Titans won by 5 wickets",
    "Gujarat Titans won by 7 wickets",
    "Gujarat Titans won by 3 wickets"
  ],
  "playerAvailability": "No injury updates available.",
  "powerplaySpecialists": [
    "Ishant Sharma picked 9 wickets in the powerplay phase.",
    "Ravichandran Ashwin picked 4 wickets in the powerplay phase.",
    "Kagiso Rabada picked 13 wickets in the powerplay phase.",
    "Prasidh Krishna

In [12]:
import json
from typing import Dict, List, Any

class CricketMatchAnalyzer:
    def __init__(self, player_data: Dict, match_summary: Dict):
        self.player_data = player_data
        self.match_summary = match_summary['match_summary']  # Access nested match_summary
        
    def analyze_recent_form(self, player_id: str) -> Dict:
        """Analyze player's recent form from last few matches"""
        try:
            player = self.player_data[str(player_id)]  # Convert player_id to string
            recent_form = {
                'batting': {'runs': 0, 'balls': 0, 'strike_rate': 0},
                'bowling': {'wickets': 0, 'economy': 0, 'overs': 0}
            }
            
            if 'batting' in player and 'matches' in player['batting']:
                recent_matches = player['batting']['matches'][:5]  # Last 5 matches
                total_runs = sum(match['runs'] for match in recent_matches)
                total_balls = sum(match['balls'] for match in recent_matches)
                strike_rate = (total_runs / total_balls * 100) if total_balls > 0 else 0
                
                recent_form['batting'] = {
                    'runs': total_runs,
                    'balls': total_balls,
                    'strike_rate': round(strike_rate, 2)
                }
                
            if 'bowling' in player and 'matches' in player['bowling']:
                recent_matches = player['bowling']['matches'][:5]  # Last 5 matches
                total_wickets = sum(match['wickets'] for match in recent_matches)
                total_overs = sum(float(match['overs']) for match in recent_matches)
                total_runs = sum(match['runs'] for match in recent_matches)
                economy = (total_runs / total_overs) if total_overs > 0 else 0
                
                recent_form['bowling'] = {
                    'wickets': total_wickets,
                    'economy': round(economy, 2),
                    'overs': round(total_overs, 1)
                }
            
            return recent_form
        except Exception as e:
            print(f"Error analyzing recent form for player {player_id}: {str(e)}")
            return recent_form

    def analyze_venue_stats(self) -> Dict:
        """Analyze venue statistics"""
        try:
            venue_stats = self.match_summary['venue_stats']
            
            # Calculate aggregated stats for pace and spin
            pace_stats = [stat for stat in venue_stats if stat['bowler_type'] == 'pace']
            spin_stats = [stat for stat in venue_stats if stat['bowler_type'] == 'spin']
            
            # Average economy rates
            pace_economy = sum(stat['economy_rate'] for stat in pace_stats) / len(pace_stats)
            spin_economy = sum(stat['economy_rate'] for stat in spin_stats) / len(spin_stats)
            
            # Determine pitch nature
            pitch_nature = {
                'favors_pace': pace_economy < spin_economy,
                'high_scoring': pace_economy > 8.5 or spin_economy > 8.5,
                'average_score': sum(stat['total_runs'] for stat in venue_stats) / 20
            }
            
            return {
                'pace_stats': {
                    'economy': round(pace_economy, 2),
                    'wickets_per_match': sum(stat['total_wickets'] for stat in pace_stats) / len(pace_stats)
                },
                'spin_stats': {
                    'economy': round(spin_economy, 2),
                    'wickets_per_match': sum(stat['total_wickets'] for stat in spin_stats) / len(spin_stats)
                },
                'pitch_nature': pitch_nature
            }
        except Exception as e:
            print(f"Error analyzing venue stats: {str(e)}")
            return {
                'pace_stats': {'economy': 0, 'wickets_per_match': 0},
                'spin_stats': {'economy': 0, 'wickets_per_match': 0},
                'pitch_nature': {'favors_pace': False, 'high_scoring': False, 'average_score': 0}
            }

    def analyze_phase_stats(self) -> Dict:
        """Analyze player performance in different phases"""
        phase_specialists = {
            'powerplay': {'batsmen': [], 'bowlers': []},
            'death': {'batsmen': [], 'bowlers': []}
        }
        
        try:
            for player_id, player in self.player_data.items():
                if 'batting' in player and 'phase_stats' in player['batting']:
                    batting_stats = player['batting']['phase_stats']
                    
                    # Powerplay batting analysis
                    if batting_stats['powerplay']['balls'] > 20:  # Minimum 20 balls faced
                        powerplay_sr = batting_stats['powerplay']['runs'] / batting_stats['powerplay']['balls'] * 100
                        if powerplay_sr > 130:
                            phase_specialists['powerplay']['batsmen'].append({
                                'name': player['name'],
                                'strike_rate': round(powerplay_sr, 2)
                            })
                    
                    # Death overs batting analysis
                    if batting_stats['death']['balls'] > 15:  # Minimum 15 balls faced
                        death_sr = batting_stats['death']['runs'] / batting_stats['death']['balls'] * 100
                        if death_sr > 150:
                            phase_specialists['death']['batsmen'].append({
                                'name': player['name'],
                                'strike_rate': round(death_sr, 2)
                            })
                
                if 'bowling' in player and 'phase_stats' in player['bowling']:
                    bowling_stats = player['bowling']['phase_stats']
                    
                    # Convert balls to overs for calculation
                    powerplay_overs = bowling_stats['powerplay']['balls'] / 6
                    death_overs = bowling_stats['death']['balls'] / 6
                    
                    if powerplay_overs >= 2:  # Minimum 2 overs
                        powerplay_economy = bowling_stats['powerplay']['runs'] / powerplay_overs
                        if powerplay_economy < 8:
                            phase_specialists['powerplay']['bowlers'].append({
                                'name': player['name'],
                                'economy': round(powerplay_economy, 2)
                            })
                    
                    if death_overs >= 2:  # Minimum 2 overs
                        death_economy = bowling_stats['death']['runs'] / death_overs
                        if death_economy < 10:
                            phase_specialists['death']['bowlers'].append({
                                'name': player['name'],
                                'economy': round(death_economy, 2)
                            })
            
            return phase_specialists
        except Exception as e:
            print(f"Error analyzing phase stats: {str(e)}")
            return phase_specialists

    def get_captain_suggestions(self) -> Dict:
        """Suggest captain and vice-captain based on recent form"""
        player_scores = []
        
        try:
            for player_id, player in self.player_data.items():
                recent_form = self.analyze_recent_form(player_id)
                score = 0
                
                # Batting score calculation
                batting_stats = recent_form['batting']
                score += batting_stats['runs'] * 0.5  # 0.5 points per run
                if batting_stats['strike_rate'] > 140:
                    score += 10  # Bonus for high strike rate
                
                # Bowling score calculation
                bowling_stats = recent_form['bowling']
                score += bowling_stats['wickets'] * 20  # 20 points per wicket
                if bowling_stats['economy'] > 0 and bowling_stats['economy'] < 7:
                    score += 10  # Bonus for good economy
                
                # Add role-based bonus
                role_bonus = {
                    'Bowling Allrounder': 15,
                    'Batting Allrounder': 15,
                    'Batsman': 10,
                    'Bowler': 10
                }
                score += role_bonus.get(player.get('role', ''), 0)
                
                player_scores.append({
                    'name': player['name'],
                    'score': score,
                    'role': player['role'],
                    'recent_performance': {
                        'batting': batting_stats,
                        'bowling': bowling_stats
                    }
                })
            
            # Sort by score
            player_scores.sort(key=lambda x: x['score'], reverse=True)
            
            return {
                'captain': player_scores[0] if player_scores else None,
                'vice_captain': player_scores[1] if len(player_scores) > 1 else None,
                'differential_pick': player_scores[2] if len(player_scores) > 2 else None
            }
        except Exception as e:
            print(f"Error getting captain suggestions: {str(e)}")
            return {'captain': None, 'vice_captain': None, 'differential_pick': None}

    def generate_insights(self) -> Dict:
        """Generate comprehensive match insights"""
        try:
            venue_analysis = self.analyze_venue_stats()
            phase_specialists = self.analyze_phase_stats()
            captain_suggestions = self.get_captain_suggestions()
            
            toss_analysis = [match['winner'] for match in self.match_summary['venue_analysis']]
            batting_first_wins = sum(1 for result in toss_analysis if result == 'Batting Team')
            total_matches = len(toss_analysis)
            
            insights = {
                'venue_analysis': {
                    'pitch_nature': 'High scoring' if venue_analysis['pitch_nature']['high_scoring'] else 'Balanced',
                    'favors': 'Pace bowlers' if venue_analysis['pitch_nature']['favors_pace'] else 'Spin bowlers',
                    'average_score': round(venue_analysis['pitch_nature']['average_score']),
                    'batting_first_advantage': f"{(batting_first_wins/total_matches)*100:.1f}%"
                },
                'phase_specialists': phase_specialists,
                'captain_picks': captain_suggestions,
                'head_to_head': self.match_summary['head_to_head_analysis'][:3]  # Last 3 matches
            }
            
            return insights
        except Exception as e:
            print(f"Error generating insights: {str(e)}")
            return {}

def format_insights(insights: Dict) -> str:
    """Format insights into readable text"""
    if not insights:
        return "Unable to generate insights due to insufficient data."
    
    output = "🏏 QUICK KEY INSIGHTS FOR FANTASY TEAM 🏏\n\n"
    
    # Venue Analysis
    output += "1. PITCH AND VENUE ANALYSIS:\n"
    output += f"• Pitch Nature: {insights['venue_analysis']['pitch_nature']}\n"
    output += f"• Favors: {insights['venue_analysis']['favors']}\n"
    output += f"• Average Score: {insights['venue_analysis']['average_score']}\n"
    output += f"• Batting First Win %: {insights['venue_analysis']['batting_first_advantage']}\n\n"
    
    # Phase Specialists
    output += "2. PHASE SPECIALISTS:\n"
    if insights['phase_specialists']['powerplay']['batsmen']:
        output += "Powerplay Batsmen:\n"
        for batsman in insights['phase_specialists']['powerplay']['batsmen'][:2]:
            output += f"• {batsman['name']} (SR: {batsman['strike_rate']})\n"
    
    if insights['phase_specialists']['death']['bowlers']:
        output += "\nDeath Bowlers:\n"
        for bowler in insights['phase_specialists']['death']['bowlers'][:2]:
            output += f"• {bowler['name']} (Economy: {bowler['economy']})\n\n"
    
    # Captain Picks
    output += "3. FANTASY PICKS:\n"
    if insights['captain_picks']['captain']:
        captain = insights['captain_picks']['captain']
        output += f"• Captain: {captain['name']} ({captain['role']})\n"
        
    if insights['captain_picks']['vice_captain']:
        vc = insights['captain_picks']['vice_captain']
        output += f"• Vice Captain: {vc['name']} ({vc['role']})\n"
        
    if insights['captain_picks']['differential_pick']:
        diff = insights['captain_picks']['differential_pick']
        output += f"• Differential Pick: {diff['name']} ({diff['role']})\n\n"
    
    # Head to Head
    output += "4. RECENT HEAD TO HEAD:\n"
    for result in insights['head_to_head']:
        output += f"• {result}\n"
    
    return output

def analyze_match(player_data: Dict, match_summary: Dict) -> str:
    """Main function to analyze match and generate insights"""
    try:
        # Create analyzer instance
        analyzer = CricketMatchAnalyzer(player_data, match_summary)
        
        # Generate and format insights
        insights = analyzer.generate_insights()
        formatted_insights = format_insights(insights)
        
        return formatted_insights
    except Exception as e:
        return f"Error analyzing match: {str(e)}"

# Example usage
if __name__ == "__main__":
    # Your data is already loaded in player_data and match_summary variables
    insights = analyze_match(player_data, venue_data)
    print(insights)


Error analyzing phase stats: 'powerplay'
Error getting captain suggestions: 'role'
🏏 QUICK KEY INSIGHTS FOR FANTASY TEAM 🏏

1. PITCH AND VENUE ANALYSIS:
• Pitch Nature: High scoring
• Favors: Spin bowlers
• Average Score: 543
• Batting First Win %: 50.0%

2. PHASE SPECIALISTS:
Powerplay Batsmen:
• Ravichandran Ashwin (SR: 144.12)
• Jos Buttler (SR: 131.48)

Death Bowlers:
• Ravichandran Ashwin (Economy: 8.53)

• Kagiso Rabada (Economy: 9.49)

3. FANTASY PICKS:
4. RECENT HEAD TO HEAD:
• Chennai Super Kings won by 63 runs
• Chennai Super Kings won by 5 wickets
• Chennai Super Kings won by 15 runs



In [18]:
import pandas as pd
import json
from typing import Dict, List, Any
from datetime import datetime
import numpy as np
from collections import defaultdict

class EnhancedCricketAnalyzer:
    def __init__(self, player_data: Dict, venue_data: Dict):
        self.player_data = player_data
        self.venue_data = venue_data
        self.recent_matches_count = 5
        
    def calculate_player_form(self, player_data: Dict, match_type: str = 'recent') -> Dict:
        """Calculate detailed player form statistics"""
        batting_stats = defaultdict(int)
        bowling_stats = defaultdict(int)
        
        if 'batting' in player_data and 'matches' in player_data['batting']:
            matches = [m for m in player_data['batting']['matches'] if m['stats_type'] == match_type]
            if matches:
                batting_stats.update({
                    'total_runs': sum(m['runs'] for m in matches),
                    'total_balls': sum(m['balls'] for m in matches),
                    'total_matches': len(matches),
                    'fifties': sum(1 for m in matches if m['runs'] >= 50),
                    'hundreds': sum(1 for m in matches if m['runs'] >= 100),
                    'avg_position': sum(m.get('batting_position', 11) for m in matches) / len(matches),
                    'strike_rate': sum(m['runs'] for m in matches) / sum(m['balls'] for m in matches) * 100 if sum(m['balls'] for m in matches) > 0 else 0,
                    'fantasy_points': sum(m['fantasy_points'] for m in matches)
                })
        
        if 'bowling' in player_data and 'matches' in player_data['bowling']:
            matches = [m for m in player_data['bowling']['matches'] if m['stats_type'] == match_type]
            if matches:
                bowling_stats.update({
                    'wickets': sum(m['wickets'] for m in matches),
                    'overs': sum(m['overs'] for m in matches),
                    'runs_conceded': sum(m['runs'] for m in matches),
                    'economy': sum(m['runs'] for m in matches) / sum(m['overs'] for m in matches) if sum(m['overs'] for m in matches) > 0 else 0,
                    'fantasy_points': sum(m['fantasy_points'] for m in matches),
                    'matches': len(matches)
                })
        
        return {
            'batting': dict(batting_stats),
            'bowling': dict(bowling_stats),
            'name': player_data.get('name', ''),
            'role': player_data.get('role', ''),
            'team': player_data.get('team_name', '')
        }

    def get_top_performers(self) -> Dict:
        """Get top performers in various categories"""
        top_performers = {
            'batsmen': [],
            'bowlers': [],
            'all_rounders': [],
            'form_players': []
        }
        
        for player_id, data in self.player_data.items():
            form = self.calculate_player_form(data)
            
            # Batting analysis
            if form['batting'].get('total_runs', 0) > 100:
                top_performers['batsmen'].append({
                    'name': form['name'],
                    'runs': form['batting']['total_runs'],
                    'strike_rate': round(form['batting']['strike_rate'], 2),
                    'fantasy_points': form['batting']['fantasy_points']
                })
                
            # Bowling analysis
            if form['bowling'].get('wickets', 0) >= 3:
                top_performers['bowlers'].append({
                    'name': form['name'],
                    'wickets': form['bowling']['wickets'],
                    'economy': round(form['bowling']['economy'], 2),
                    'fantasy_points': form['bowling']['fantasy_points']
                })
                
            # All-rounder analysis
            if (form['batting'].get('total_runs', 0) > 50 and 
                form['bowling'].get('wickets', 0) >= 2):
                top_performers['all_rounders'].append({
                    'name': form['name'],
                    'runs': form['batting']['total_runs'],
                    'wickets': form['bowling']['wickets'],
                    'fantasy_points': form['batting']['fantasy_points'] + form['bowling']['fantasy_points']
                })
                
            # Form players (based on fantasy points)
            total_fantasy_points = (form['batting'].get('fantasy_points', 0) + 
                                  form['bowling'].get('fantasy_points', 0))
            if total_fantasy_points > 100:
                top_performers['form_players'].append({
                    'name': form['name'],
                    'fantasy_points': total_fantasy_points,
                    'role': form['role']
                })
        
        # Sort all lists by relevant metrics
        top_performers['batsmen'].sort(key=lambda x: x['fantasy_points'], reverse=True)
        top_performers['bowlers'].sort(key=lambda x: x['fantasy_points'], reverse=True)
        top_performers['all_rounders'].sort(key=lambda x: x['fantasy_points'], reverse=True)
        top_performers['form_players'].sort(key=lambda x: x['fantasy_points'], reverse=True)
        
        return top_performers

    def analyze_venue_trends(self) -> Dict:
        """Analyze venue trends and patterns"""
        venue_stats = self.venue_data['match_summary']['venue_stats']
        toss_analysis = self.venue_data['match_summary']['venue_analysis']
        
        # Analyze toss impact
        toss_decisions = [match['toss_decision'] for match in toss_analysis]
        toss_winners = [match['winner'] for match in toss_analysis]
        
        venue_trends = {
            'pitch_behavior': {
                'pace_bowling': next((stat for stat in venue_stats 
                                    if stat['innings'] == 'Overall' and 
                                    stat['bowler_type'] == 'pace'), {}),
                'spin_bowling': next((stat for stat in venue_stats 
                                    if stat['innings'] == 'Overall' and 
                                    stat['bowler_type'] == 'spin'), {})
            },
            'toss_analysis': {
                'batting_first_wins': sum(1 for w in toss_winners if w == 'Batting Team'),
                'bowling_first_wins': sum(1 for w in toss_winners if w == 'Bowling Team'),
                'preferred_choice': max(set(toss_decisions), key=toss_decisions.count)
            },
            'scoring_patterns': {
                'first_innings': next((stat for stat in venue_stats 
                                     if stat['innings'] == 'Inning 1' and 
                                     stat['bowler_type'] == 'pace'), {}),
                'second_innings': next((stat for stat in venue_stats 
                                      if stat['innings'] == 'Inning 2' and 
                                      stat['bowler_type'] == 'pace'), {})
            }
        }
        
        return venue_trends

    def get_phase_specialists(self) -> Dict:
        """Identify specialists for different phases of the game"""
        specialists = {
            'powerplay': {'batsmen': [], 'bowlers': []},
            'middle': {'batsmen': [], 'bowlers': []},
            'death': {'batsmen': [], 'bowlers': []}
        }

        for player_id, data in self.player_data.items():
            # Batting phase analysis
            if 'batting' in data and 'phase_stats' in data['batting']:
                for phase in ['powerplay', 'middle', 'death']:
                    phase_stats = data['batting']['phase_stats'].get(phase)  # Safely get phase stats
                    if phase_stats and phase_stats.get('balls', 0) > 20:  # Ensure stats exist and meet the criteria
                        strike_rate = phase_stats.get('strike_rate', 0)
                        if strike_rate > 150:  # High strike rate threshold
                            specialists[phase]['batsmen'].append({
                                'name': data['name'],
                                'strike_rate': round(strike_rate, 2),
                                'runs': phase_stats.get('runs', 0)
                            })

            # Bowling phase analysis
            if 'bowling' in data and 'phase_stats' in data['bowling']:
                for phase in ['powerplay', 'middle', 'death']:
                    phase_stats = data['bowling']['phase_stats'].get(phase)  # Safely get phase stats
                    if phase_stats and phase_stats.get('balls', 0) > 24:  # Ensure stats exist and meet the criteria
                        economy = (phase_stats.get('runs', 0) * 6) / phase_stats.get('balls', 1)  # Avoid division by zero
                        if economy < 8:  # Good economy threshold
                            specialists[phase]['bowlers'].append({
                                'name': data['name'],
                                'economy': round(economy, 2),
                                'wickets': phase_stats.get('wickets', 0)
                            })
 
        # Sort specialists by their respective metrics
        for phase in specialists:
            specialists[phase]['batsmen'].sort(key=lambda x: x['strike_rate'], reverse=True)
            specialists[phase]['bowlers'].sort(key=lambda x: x['economy'])
            
        return specialists

    def generate_captain_picks(self) -> Dict:
        """Generate captain and vice-captain recommendations"""
        player_scores = []
        
        for player_id, data in self.player_data.items():
            form = self.calculate_player_form(data)
            
            # Calculate weighted score based on recent performance
            score = (
                form['batting'].get('fantasy_points', 0) * 0.6 +  # 60% weight to batting
                form['bowling'].get('fantasy_points', 0) * 0.4    # 40% weight to bowling
            )
            
            if score > 0:
                player_scores.append({
                    'name': form['name'],
                    'role': form['role'],
                    'score': score,
                    'form_rating': 'Excellent' if score > 200 else 'Good' if score > 100 else 'Average',
                    'batting_points': form['batting'].get('fantasy_points', 0),
                    'bowling_points': form['bowling'].get('fantasy_points', 0)
                })
        
        player_scores.sort(key=lambda x: x['score'], reverse=True)
        
        return {
            'captain_picks': player_scores[:3],  # Top 3 captain options
            'differential_picks': [p for p in player_scores[3:8] if p['score'] > 50]  # Next 5 good performers
        }

    def generate_complete_insights(self) -> Dict:
        """Generate comprehensive match insights"""
        top_performers = self.get_top_performers()
        venue_trends = self.analyze_venue_trends()
        phase_specialists = self.get_phase_specialists()
        captain_picks = self.generate_captain_picks()
        
        insights = {
            'match_insights': {
                'top_performers': top_performers,
                'venue_analysis': venue_trends,
                'phase_specialists': phase_specialists,
                'captain_recommendations': captain_picks,
                'head_to_head': self.venue_data['match_summary']['head_to_head_analysis'],
                'generated_at': datetime.now().strftime("%Y-%m-%d %H:%M:%S")
            },
            'fantasy_tips': [
                "Pick players batting in top 4 for this high-scoring venue",
                f"Prefer {venue_trends['toss_analysis']['preferred_choice']} first specialists based on toss analysis",
                "Include at least 2 all-rounders in your team",
                "Pick death over specialists for maximum points",
                "Consider pitch behavior while selecting spinner vs pacer ratio"
            ]
        }
        
        return insights

def save_insights_to_file(insights: Dict, filename: str = 'match_insights.json'):
    """Save insights to a JSON file"""
    with open(filename, 'w') as f:
        json.dump(insights, f, indent=2)
    print(f"Insights saved to {filename}")

def main(player_data_file: str, venue_data_file: str):
    """Main function to run the analysis"""
    # Load data
    with open(player_data_file, 'r') as f:
        player_data = json.load(f)
    with open(venue_data_file, 'r') as f:
        venue_data = json.load(f)
    
    # Create analyzer instance
    analyzer = EnhancedCricketAnalyzer(player_data, venue_data)
    
    # Generate insights
    insights = analyzer.generate_complete_insights()
    
    # Save insights to file
    save_insights_to_file(insights)
    
    return insights

# Example usage
if __name__ == "__main__":
    insights = main("playerWiseData.json", "processedData.json")
    print("Analysis complete! Check match_insights.json for detailed insights.")

Insights saved to match_insights.json
Analysis complete! Check match_insights.json for detailed insights.
